In [ ]:
import sympy as sp
import numpy as np

# Armijo para obtener lambda
def armijo_line_search(func, gradient, xk, direction, lamb = 0.5, c1 = 0.9, max_iter = 100):
    """
    Armijo line search to find a suitable step size.

    Parameters:
    - func: The objective function to minimize.
    - gradient: The gradient of the objective function.
    - x: Current iterate.
    - direction: Search direction.
    - lamb: Initial step size.
    - c1: Armijo condition parameter.
    - max_iter: Maximum number of iterations.

    Returns:
    - lamb: The step size that satisfies Armijo-Wolfe conditions.
    """
    def armijo_condition(lamb):
        x_new = xk + lamb * direction
        func_value = func(*x_new)  # Evaluate the symbolic function at x_new
        return (func_value <= func(*xk) + c1 * lamb * gradient.dot(direction))
    
    for _ in range(max_iter):
        if armijo_condition(lamb):
            break
        lamb = lamb / 2  # Reduce step size if conditions are not met

    return lamb  # Return the best step size found

# Test de convergencia de descenso lineal
def test_convergence_linear_descend(x0, x_new, func, gradient_point_new, tol1, tol2, tol3):

    conditions = (np.linalg.norm(x0 - x_new) < tol1,
                  np.linalg.norm(gradient_point_new) < tol2,
                  abs(func(*x_new) - func(*x0)) < tol3)
    
    test = any(conditions)
    condition_idx = np.where(conditions)[0]

    return test, condition_idx

# Método del gradiente
def gradient_linear_descend(f, x, x0, tol1 = 1e-4, tol2 = 1e-4, tol3 = 1e-4, max_iterations = 100):
    
    func = sp.lambdify(x, f, 'numpy')

    gradient = [sp.diff(f, xi) for xi in x]
    gradient_func = sp.lambdify(x, gradient, 'numpy')
    
    gradient_point = gradient_func(*x0)
    gradient_point = np.array(gradient_point)
    d0 = -gradient_point
    
    # Inicialización de variables para el bucle
    iteration = 0
    
    while iteration < max_iterations:
        print(x0)
        print(iteration)
        
        lamb = armijo_line_search(func, gradient_point, x0, d0)
        
        x_new = x0 + lamb * d0 # Aquí se obtiene el x_{k + 1}
        gradient_point_new = np.array(gradient_func(*x_new))
        d_new = -gradient_point_new
        
        test, cond_idx = test_convergence_linear_descend(x0, x_new, func, gradient_point_new, tol1, tol2, tol3)
        if test:
            print(f"Convergence reached in iteration {iteration}. Fullfilled conditions: {cond_idx}")
            break
        else:
            x0 = x_new
            d0 = d_new
            gradient_point = gradient_point_new
            iteration = iteration + 1

    return x_new, iteration
    
# Llamada a función

x, y = sp.symbols('x y')
f = 9*x**2 + 2*x*y + y**2

initial_guess = [0, -1]

result, iterations = gradient_linear_descend(f, (x, y), initial_guess)
print(result)
print(iterations)

In [ ]:
import sympy as sp
import numpy as np

def newtons_method(f, x, initial_guess, tol = 1e-6, max_iter = 100):
    
    """
    Newton's method for finding the minimum of a multivariable function.
    
    Parameters:
    f: function to minimize
    x: list of sympy symbols
    initial_guess: initial point to iterate
    tol: float, optional
        Tolerance for stopping criterion. The iteration stops when the norm
        of the gradient is less than tol.
    max_iter: int, optional
        Maximum number of iterations.
        
    Returns:
    x_0: array
        The estimated minimum point as a list of numerical values.
    iteration: int
        The number of iterations performed.
    """
    gradient = [sp.diff(f, xi) for xi in x]
    hessian = sp.hessian(f, x)
    gradient_func = sp.lambdify(x, gradient, 'numpy')
    hessian_func = sp.lambdify(x, hessian, 'numpy')

    x0 = initial_guess
    
    for iteration in range(max_iter):
        print(x0)
        print(iteration)
        grad = gradient_func(*x0)
        hess = hessian_func(*x0)
        step = -np.dot(np.linalg.inv(hess), grad)
        x0 = x0 + step
        
        if np.linalg.norm(grad) < tol:
            break

    return x0, iteration


# Llamada a función

x, y = sp.symbols('x y')
f = 3*x**2 - 2*x*y + y**2 + x

initial_guess = [1, 1]

result, iterations = newtons_method(f, (x, y), initial_guess)
print(result)
print(iterations)

In [12]:
import math

def golden_ratio_search(func, lower, upper, tolerance):

    golden_ratio = (math.sqrt(5) - 1) / 2

    # Calculate initial values for the search
    a = lower
    b = upper
    x1 = a + (1 - golden_ratio) * (b - a)
    x2 = a + golden_ratio * (b - a)

    iter = 0
    
    while abs(b - a) > tolerance:
        if func(x1) < func(x2):
            b = x2 # a no se ve modificado
        else:
            a = x1 # b no se ve modificado

        x1 = a + (1 - golden_ratio) * (b - a)
        x2 = a + golden_ratio * (b - a)
        iter = iter + 1

    # Return the approximate minimum point and value
    min_point = (a + b) / 2
    min_value = func(min_point)

    return min_point, min_value, iter


def test_function(x):
    return -x * math.cos(x)

lower_bound = 0
upper_bound = math.pi/2
tolerance = 1e-6

min_point, min_value, iter = golden_ratio_search(test_function, lower_bound, upper_bound, tolerance)
print(f"Punto en el mínimo: {min_point}")
print(f"Valor de la función en el mínimo: {min_value}")
print(f"Número de iteraciones: {iter}")

Punto en el mínimo: 0.8603334800557735
Valor de la función en el mínimo: -0.5610963381910328
Número de iteraciones: 30


In [30]:
import sympy as sp

def newton_optimization_1d(f, x0, tol = 1e-6, max_iter = 100):
    
    x = sp.symbols('x')
    
    f_expr = sp.sympify(f)
    
    f_prime = f_expr.diff(x)
    f_double_prime = f_prime.diff(x)
    
    x_current = x0
    iteration = 0
    
    while iteration < max_iter:
        df_prime = f_prime.subs(x, x_current)
        df_double_prime = f_double_prime.subs(x, x_current)
        
        x_new = x_current - df_prime / df_double_prime # iteración
        
        if abs(x_new - x_current) < tol: # salida por tolerancia
            
            second_derivative = df_double_prime.subs(x, x_new)
            
            if second_derivative > 0:
                print(f"Minimum found at x = {x_new}")
            elif second_derivative < 0:
                print(f"Maximum found at x = {x_new}")
            else:
                print(f"Saddle point found at x = {x_new}")
                
            return x_new, iteration
        
        x_current = x_new
        iteration += 1
        
    return x_current, iteration

# main 

f = 'x ** 4 - 14*x**3 + 60*x**2 - 70*x'
x0 = 0.7 # valor inicial
    
result, iter = newton_optimization_1d(f, x0, tol = 0.001, max_iter = 100)

print(f"El mínimo está en: {result}")
print(f"Number of iterations:", {iter})

Minimum found at x = 0.780884053071772
El mínimo está en: 0.780884053071772
Number of iterations: {2}


In [33]:
import sympy as sp

def bisect_minimum(func, a, b, tol=1e-6, max_iter=100):
    """
    Find the minimum of a function within the interval [a, b] using derivative information.

    Parameters:
    func (SymPy expression): The symbolic expression for the function.
    a (float): Left endpoint of the interval.
    b (float): Right endpoint of the interval.
    tol (float): Tolerance for the minimum approximation.
    max_iter (int): Maximum number of iterations.

    Returns:
    float: Approximation of the minimum.
    """

    x = sp.symbols('x')
    derivative = sp.diff(func, x)
    
    # Convert the symbolic functions to lambdas for numerical evaluation
    func_lambda = sp.lambdify(x, func, 'numpy')
    derivative_lambda = sp.lambdify(x, derivative, 'numpy')

    if derivative_lambda(a) * derivative_lambda(b) >= 0:
        raise ValueError("Derivative values at endpoints must have different signs.")

    iteration = 0

    while (b - a) / 2 > tol and iteration < max_iter:
        c = (a + b) / 2
        if derivative_lambda(c) == 0:
            return c
        elif derivative_lambda(c) * derivative_lambda(a) < 0:
            b = c
        else:
            a = c

        iteration += 1

    return (a + b) / 2

# Example usage:
if __name__ == "__main__":
    x = sp.symbols('x')
    func = x**4 - 14*x**3 + 60*x**2 - 70*x  # A simple quadratic function with a minimum at x = -1

    minimum = bisect_minimum(func, 0, 2)
    print("Approximate minimum:", minimum)


Approximate minimum: 0.7808847427368164


In [ ]:
import numpy as np

def conjugate_gradient(A, b, x0, tol = 1e-6, max_iter = 100):
    
    x = x0
    r = np.dot(A, x) - b 
    d = -r
    
    for iteration in range(max_iter):
        print(x)
        
        Ad = np.dot(A, d)
        lamb = np.dot(r, r) / np.dot(d, Ad)
        x = x + lamb * d
        r_new = np.dot(A, x) - b
        
        if np.linalg.norm(r_new) < tol:
            print(x)
            break
        
        beta = np.dot(r_new, r_new) / np.dot(r, r)
        d = -r_new + beta * d
        r = r_new
        
    return x, iteration

# Example usage
if __name__ == "__main__":
    A = np.array([[4, -1, 1], [-1, 4, -2], [1, -2, 4]]) # entre corchetes por filas
    b = np.array([12, -1, 5])
    x0 = np.array([0, 0, 0])

    solution, iterations = conjugate_gradient(A, b, x0)
    print("Solution:", solution)
    print("iteraciones", iterations)


In [38]:
import sympy as sp
import numpy as np

def armijo_line_search(func, gradient, xk, direction, lamb = 1, c1 = 0.6, max_iter = 100):
    """
    Armijo line search to find a suitable step size.

    Parameters:
    - func: The objective function to minimize.
    - gradient: The gradient of the objective function.
    - x: Current iterate.
    - direction: Search direction.
    - lamb: Initial step size.
    - c1: Armijo condition parameter.
    - max_iter: Maximum number of iterations.

    Returns:
    - lamb: The step size that satisfies Armijo-Wolfe conditions.
    """
    def armijo_condition(lamb):
        x_new = xk + lamb * direction
        func_value = func(*x_new)  # Evaluate the symbolic function at x_new
        return (func_value <= func(*xk) + c1 * lamb * gradient.dot(direction))
    
    for _ in range(max_iter):
        if armijo_condition(lamb):
            break
        lamb = lamb / 2  # Reduce step size if conditions are not met

    return lamb  # Return the best step size found

def fletcher_reeves(f, x, x0, tol = 1e-4, max_iterations = 100):
    
    gradient = [sp.diff(f, xi) for xi in x]
    func = sp.lambdify(x, f, 'numpy')
    gradient_func = sp.lambdify(x, gradient, 'numpy')
    
    gradient_point = gradient_func(*x0)
    gradient_point = np.array(gradient_point)
    d0 = -gradient_point
    
    # Inicialización de variables para el bucle
    iteration = 0
    x_new = x0
    d_new = d0
    gradient_point_new = gradient_point
    
    while (np.linalg.norm(gradient_point) > tol and iteration < max_iterations):
        
        lamb = armijo_line_search(func, gradient_point_new, x_new, d_new)
        
        gradient_point_old = gradient_func(*x_new) # Este es el gradiente evaluado en el x_k
        
        x_new = x_new + lamb*d_new # Aquí se pone el x_{k + 1}
        
        gradient_point_new = gradient_func(*x_new)
        gradient_point_new = np.array(gradient_point_new)
        
        beta_new = np.dot(gradient_point_new, gradient_point_new) / np.dot(gradient_point_old, gradient_point_old)
        d_new = -gradient_point_new + beta_new * d_new
        
        iteration = iteration + 1
    
    return x_new, iteration
    
# Llamada a función

x, y = sp.symbols('x y')
f = (2*x - y)**2 + (y + 1)**2

initial_guess = [5/2, 2]

result, iterations = fletcher_reeves(f, (x, y), initial_guess)
print(result)
print(iterations)

[-0.5 -1. ]
100


In [3]:
import sympy as sp
import numpy as np
import pandas as pd

def armijo_line_search(func, gradient, xk, direction, lamb = 0.5, c1 = 0.9, max_iter = 100):
    """
    Armijo line search to find a suitable step size.

    Parameters:
    - func: The objective function to minimize.
    - gradient: The gradient of the objective function.
    - x: Current iterate.
    - direction: Search direction.
    - lamb: Initial step size.
    - c1: Armijo condition parameter.
    - max_iter: Maximum number of iterations.

    Returns:
    - lamb: The step size that satisfies Armijo-Wolfe conditions.
    """
    def armijo_condition(lamb):
        x_new = xk + lamb * direction
        func_value = func(*x_new)  # Evaluate the symbolic function at x_new
        return (func_value <= func(*xk) + c1 * lamb * gradient.dot(direction))
    
    for _ in range(max_iter):
        if armijo_condition(lamb):
            break
        lamb = lamb / 2  # Reduce step size if conditions are not met

    return lamb  # Return the best step size found

def fletcher_reeves(f, x, x0, tol = 1e-3, max_iterations = 100):
    results = []
    
    gradient = [sp.diff(f, xi) for xi in x]
    func = sp.lambdify(x, f, 'numpy')
    gradient_func = sp.lambdify(x, gradient, 'numpy')
    
    gradient_point = gradient_func(*x0)
    gradient_point = np.array(gradient_point)
    d0 = -gradient_point
    
    # Inicialización de variables para el bucle
    iteration = 0
    x_new = x0
    d_new = d0
    gradient_point_new = gradient_point
    results.append([iteration, np.round(x_new, 5)])
    
    while (np.linalg.norm(gradient_point) > tol and iteration < max_iterations):
        
        lamb = armijo_line_search(func, gradient_point_new, x_new, d_new)
        
        gradient_point_old = gradient_func(*x_new) # Este es el gradiente evaluado en el x_k
        
        x_new = x_new + lamb*d_new # Aquí se pone el x_{k + 1}
        
        gradient_point_new = gradient_func(*x_new)
        gradient_point_new = np.array(gradient_point_new)
        
        beta_new = np.dot(gradient_point_new, gradient_point_new) / np.dot(gradient_point_old, gradient_point_old)
        d_new = -gradient_point_new + beta_new * d_new
        
        iteration = iteration + 1
        results.append([iteration, np.round(x_new, 5)])
        
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'x']
    return table
    
# Llamada a función

x, y = sp.symbols('x y')
f = x - y + 2*x**2 + 2*x*y + y**2

initial_guess = [0, 0]

result = fletcher_reeves(f, (x, y), initial_guess)
result

,Iteration,x
0,0,"[0, 0]"
1,1,"[-0.125, 0.125]"
2,2,"[-0.2207, 0.23633]"
3,3,"[-0.33636, 0.38853]"
4,4,"[-0.45221, 0.56147]"
...,...,...
96,96,"[-1.0, 1.5]"
97,97,"[-1.0, 1.5]"
98,98,"[-1.0, 1.5]"
99,99,"[-1.0, 1.5]"


In [15]:
from numpy.linalg import norm
def levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals, tol = 1e-4, max_iter = 10, lambda_0 = 10):
    
    # Obtenemos la lista de funciones evaluados en todos los puntos
    # de variables independientes
    f = []
    for idx in range(len(indep_vars_values[indep_vars_symbols[0]])):
        subs_values = {key: values[idx] for key, values in indep_vars_values.items()}
        f.append(func.subs(subs_values))
    # Residuos
    r = [f_j - y_j for f_j, y_j in zip(f, y_vec)]
    # Jacobiano
    jacobo = sp.Matrix(f).jacobian(params_symbols)

    # Inicializamos variables para el bucle
    results = []
    params_k = init_vals
    lambda_k = lambda_0
    diff_cond = 2 * tol
    iter_count = 0
    last_iter = False

    while iter_count < max_iter:

        # Valores de la iteración k
        jacobo_k = jacobo.subs(params_k)
        r_k = np.asarray([r_j.subs(params_k) for r_j in r], dtype = np.float16)
        d_k = - np.dot(
            sp.Matrix(
                np.dot(jacobo_k.T, jacobo_k) +
                lambda_k * np.diag(np.dot(jacobo_k.T, jacobo_k).diagonal())).inv(),
            np.dot(jacobo_k.T, r_k))
        
        # Valores de la iteración k + 1
        params_k1 = {key_j: value_j + d_k_j for key_j, value_j, d_k_j in zip(params_k.keys(), params_k.values(), d_k)}
        r_k1 = np.asarray([r_j.subs(params_k1) for r_j in r], dtype = np.float16)

        # Añadimos la iteración a la lista de resultados
        results.append([iter_count] + list(params_k.values()) + [sum(r_k**2), lambda_k] + list(d_k))

        # Condición para el siguiente paso
        lambda_cond = norm(r_k1) ** 2 - norm(r_k)**2
        if lambda_cond > 0:
            lambda_k *= 10
        else:
            lambda_k /= 10
            params_k = params_k1
            diff_cond = lambda_cond
        iter_count += 1

        # Condición de salida
        if last_iter:
            break
        if abs(diff_cond) < tol:
            last_iter = True

    # Generamos tabla con los resultados
    table = pd.DataFrame(results)
    table.columns = ['Iter'] + [str(symbol) for symbol in params_symbols] + ['\u2211 r²', '\u03BB'] + ['d_' + str(symbol) for symbol in params_symbols]
    
    return table, params_k



# Ejemplo diapositivas

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]
indep_vars_values = {t: [0, 0.5, 1, 1.5, 2, 2.5, 3]}
y_vec = [1.145, 0.512, 0.401, 0.054, 0.038, 0.014, 0.046]

# Ecuación de ajuste
# Parámetros
x = sp.symbols('x')
params_symbols = [x]
func = np.e ** (- x * t)

# Valores iniciales para los parámetros
init_vals = {x: 4}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table


# Práctica 1: Ejercicio 2

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]
indep_vars_values = {t: [1, 2, 3, 4, 5, 6, 7, 8]}
y_vec = [8.3, 11.0, 14.7, 19.7, 26.7, 35.2, 44.4, 55.9]

# Ecuación de ajuste
# Parámetros
x1, x2 = sp.symbols('x1 x2')
params_symbols = [x1, x2]
func = x1 * np.e ** (x2 * t)

# Valores iniciales para los parámetros
init_vals = {x1: 4, x2: 0.3}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table

# Ejercicio en el tema (2D)

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]

indep_vars_values = {t: list(np.arange(-5, 5, 0.5))}
y_vec = [-0.99999, -1.2090, -1.3877, -1.5090, -1.5510, -1.5, -1.3510, -1.109, -0.7877, -0.409, 0, 0.4090, 0.787786, 1.10901,
         1.3510, 1.5, 1.5510, 1.5090, 1.3877, 1.209, 0.9999]

# Ecuación de ajuste
# Parámetros
x1, x2 = sp.symbols('x1 x2')
params_symbols = [x1, x2]
func = sp.sin(x1 * t) + x2 * t

# Valores iniciales para los parámetros
init_vals = {x1: 0.5, x2: 0.3}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table

,Iter,x1,x2,∑ r²,λ,d_x1,d_x2
0,0,0.5,0.3,4.825712e+00,10.0000,0.0221326872632251,-0.0139062009314106
1,1,0.522132687263225,0.286093799068589,3.600886e+00,1.0000,0.0779176070662545,-0.0553679622604664
2,2,0.600050294329480,0.230725836808123,4.011502e-01,0.1000,0.0285334504435690,-0.0285468002426872
3,3,0.628583744773048,0.202179036565436,6.914735e-04,0.0100,-0.000204540904874094,-0.00213121339931450
4,4,0.628379203868174,0.200047823166121,1.788139e-07,0.0010,-6.88233544527046e-5,-6.29256029316954e-5
5,5,0.628310380513722,0.199984897563190,0.000000e+00,0.0001,-3.75860888568494e-7,-2.87385368397551e-7


In [45]:
import numpy as np

# Create a NumPy array
numpy_array = np.linspace(-5, 5, num = 21)

# Convert the NumPy array to a Python list
python_list = numpy_array.tolist()

# Print the Python list
print(python_list)

print(list(np.arange(-5, 5, 0.5)))

[-5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
[-5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5]
